In [1]:
!mkdir -p ./code/config

In [2]:
%%writefile ./code/test_data_prep.py

import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import librosa
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
from joblib import delayed,Parallel
import json, random
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, re, math
from tqdm import tqdm

SR = 44100
SEC = 5
atleast_sec = 1#3
n_fft = 1024
hop_length = 512
n_mels = 128
NewMax = 255.
NewMin = 0.



folder_wav = f'./test_5sec_mel_spec'

if not os.path.exists(folder_wav):
    os.makedirs(folder_wav)


def image_creation(file,):
    
    y, sr = librosa.load(f'../input/birdclef-2022/test_soundscapes/{file}', sr = None)
    resample_y = librosa.resample(y, orig_sr=sr, target_sr=SR)
    for i in range(0, int(len(resample_y)/44100), SEC):
        
        ## lets check if the sound track is atleast greater than 3 sec
        wave = resample_y[i*SR:(i+SEC)*SR]
        
        if len(wave)> atleast_sec * SR:
            
            if len(wave)< SEC * SR:
                wave = np.concatenate([wave, np.array([0.]*abs(wave.shape[0] - SEC*SR))])  

            S = librosa.feature.melspectrogram(wave, sr=SR, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels,win_length=None)
            S_DB = librosa.power_to_db(S, ref=np.max)
            S_DB = (((S_DB - S_DB.min()) * (NewMax - NewMin)) / (S_DB.max() - S_DB.min())) + NewMin
            S_DB = np.flipud(S_DB)
            image = np.dstack([S_DB,S_DB,S_DB])
            image = image.astype(np.uint8)
            filename = file.split('.')[0]
            filename = f'{filename}'+f'_sec_{i+SEC}'
            #image = cv2.resize(image,[IMAGE_SHAPE, IMAGE_SHAPE])
            cv2.imwrite(f'{folder_wav}/{filename}.jpg',image)
            
test_sound = os.listdir('../input/birdclef-2022/test_soundscapes')

_ = Parallel(n_jobs=2, verbose=1)(
    delayed(image_creation)(
    file,
    ) for file in test_sound#[:5]
)

Writing ./code/test_data_prep.py


In [3]:
!python code/test_data_prep.py

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
code/test_data_prep.py:55: FutureWarning: Pass y=[0.1274449  0.13749163 0.10165688 ... 0.10868431 0.11853973 0.11962135] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  S = librosa.feature.melspectrogram(wave, sr=SR, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels,win_length=None)
code/test_data_prep.py:55: FutureWarning: Pass y=[ 0.11413617  0.11052517  0.11118273 ... -0.05302466 -0.04651844
 -0.0386673 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  S = librosa.feature.melspectrogram(wave, sr=SR, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels,win_length=None)
code/test_data_prep.py:55: FutureWarning: Pass y=[-0.03370828 -0.03307219 -0.03623257 ...  0.12704208  0.1257558
  0.12209577] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  S = librosa.feature.me

In [4]:
%%writefile ./code/config/config.py

class config:
    EPOCHS = 20  # 20
    BATCH_SIZE = 32
    WAVE_LENGTH = 441000
    IMAGE_SIZE = -1
    IMG_FREQ = 128
    IMG_TIME = 862
    FOLD = 7#validation fold
    WEIGHT_SAVE = "binary_v3_SED"
    N_CLASSES = 2
    IS_COLAB  = True
    DATA_LINK = "gs://kds-facb268a69ae0d4d20601c59a3b4f9215456ebf5a0246b7519d38de0"
    DATA_PATH = "data/tfrec/v3/"
    SEED = 1
    model_type = "resnet50"
    WORKERS = 6
    sample_rate = 44100
    n_fft = 1024
    win_length = None
    hop_length = 512
    n_mels = 128
    LR_START = 0.000001  # 0.000001
    LR_MAX = 0.000005  # 0.00001
    LR_MIN = 0.0000001  # 0.000001
    LR_RAMP = 4
    SAVE_DIR = "../input/bird-clief-binary-v3-sed"
    

    RESUME = False
    RESUME_EPOCH = 0
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]



Writing ./code/config/config.py


In [5]:
%%writefile ./code/utils.py

import os
import re
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
from config.config import config


AUTO = tf.data.experimental.AUTOTUNE


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    #image = tf.image.resize(image, [config.IMAGE_SIZE//2, config.IMAGE_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    # normalization
    image = image - config.mean  # [0.485, 0.456, 0.406]
    image = image / config.std  # [0.229, 0.224, 0.225]
    return image




def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    #one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.strings.split(parts[-1], '.')[0]

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    image = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    image = tf.cast(image, tf.float32) / 255.0
    # normalization
    image = image - config.mean  # [0.485, 0.456, 0.406]
    image = image / config.std  # [0.229, 0.224, 0.225]
    return image

def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def get_eval_dataset():
    list_ds = tf.data.Dataset.list_files(str('./test_5sec_mel_spec/*'), shuffle=False)
    dataset = list_ds.map(process_path)
    dataset = dataset.repeat()  # the training dataset must repeat for several epochs
    #dataset = dataset.shuffle(2048)
    dataset = dataset.batch(config.BATCH_SIZE, num_parallel_calls=AUTO, drop_remainder=False)
    dataset = dataset.prefetch(
        AUTO
    )  # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


Writing ./code/utils.py


In [6]:
%%writefile code/train_soundscape_binary_prediction.py
import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
from tqdm.auto import tqdm
import json
from datetime import datetime
from config.config import config
from utils import *
import shutil
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import gc


gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)



try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU ", tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)
config.BATCH_SIZE = config.BATCH_SIZE * strategy.num_replicas_in_sync

test_prediction = []
test_targets = []
test_file = []

## get the model for 8 fold

def model_get(model, file):
    model_list = []
    for fold in range(0,8):#8
        model_list.append(tf.keras.models.load_model(model))
        _=model_list[fold].load_weights(
        file + f'/fold_{fold}/'+ "weights/" + "best.hdf5",
    )  ## work on the path
    return model_list

#for fold in range(0,10):
## model load
K.clear_session()
model_list1 = model_get("../input/birdclif-saved-model/binary_v3_SED", "../input/bird-clief-binary-v3-sed")
model_list2 = model_get("../input/birdclif-saved-model/binary_v4_SED", "../input/bird-clief-binary-v4-sed")
model_list = model_list1 + model_list2

test_dataset = get_eval_dataset()
actual_count = len(os.listdir('./test_5sec_mel_spec'))


test_prediction_fold = []
test_targets_fold = []
test_file_fold = []
#for rr, m in enumerate(model_list):
count = 0
#print('model ',rr)
test_prediction_sub_model_0_0 = []
test_prediction_sub_model_0_1 = []
test_prediction_sub_model_0_2 = []
test_prediction_sub_model_0_3 = []
test_prediction_sub_model_0_4 = []
test_prediction_sub_model_0_5 = []
test_prediction_sub_model_0_6 = []
test_prediction_sub_model_0_7 = []

test_prediction_sub_model_1_0 = []
test_prediction_sub_model_1_1 = []
test_prediction_sub_model_1_2 = []
test_prediction_sub_model_1_3 = []
test_prediction_sub_model_1_4 = []
test_prediction_sub_model_1_5 = []
test_prediction_sub_model_1_6 = []
test_prediction_sub_model_1_7 = []
test_targets_sub = []
test_file_sub = []
for element in test_dataset:

    test_prediction_sub_model_0_0.append(tf.nn.softmax(model_list[0].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_1.append(tf.nn.softmax(model_list[1].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_2.append(tf.nn.softmax(model_list[2].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_3.append(tf.nn.softmax(model_list[3].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_4.append(tf.nn.softmax(model_list[4].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_5.append(tf.nn.softmax(model_list[5].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_6.append(tf.nn.softmax(model_list[6].__call__(element[0]))[:,1])
    test_prediction_sub_model_0_7.append(tf.nn.softmax(model_list[7].__call__(element[0]))[:,1])
    
    
    test_prediction_sub_model_1_0.append(tf.nn.softmax(model_list[8].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_1.append(tf.nn.softmax(model_list[9].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_2.append(tf.nn.softmax(model_list[10].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_3.append(tf.nn.softmax(model_list[11].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_4.append(tf.nn.softmax(model_list[12].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_5.append(tf.nn.softmax(model_list[13].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_6.append(tf.nn.softmax(model_list[14].__call__(element[0]))[:,1])
    test_prediction_sub_model_1_7.append(tf.nn.softmax(model_list[15].__call__(element[0]))[:,1])
    #test_targets_sub.append(np.argmax(element[1].numpy(),axis=1))
    test_file_sub.append(element[1].numpy())

    if count * config.BATCH_SIZE > actual_count + 20:  # this 20 is for safty
        print('the break ',count * config.BATCH_SIZE)
        break
    print('the out ',count * config.BATCH_SIZE)
    count += 1

test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_2)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_3)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_4)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_5)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_6)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_0_7)[:actual_count])

test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_2)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_3)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_4)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_5)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_6)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_7)[:actual_count])
#test_targets_fold.append(np.concatenate(test_targets_sub)[:actual_count])
test_file_fold.append(np.concatenate(test_file_sub)[:actual_count])

test_prediction = np.array(test_prediction_fold).squeeze().mean(axis=0)
#test_targets = test_targets_fold[0]
test_file = test_file_fold[0]


oof_dict = {

    'itemid':test_file,
    #'hasbird':test_targets,
    'prediction':test_prediction,
}

oof_df = pd.DataFrame.from_dict(oof_dict)
oof_df['itemid'] =  oof_df['itemid'].map(lambda x: x.decode("utf-8")) 
oof_df.to_csv(f"binary_prediction.csv", index = False)

## 5 sec
## happywhale-tfrev-train-sound-v2
## gs://kds-22c2720af882134066efd729205635efc3a6ce12dfa19de78bd41475

## 10 sec
## happywhale-tfrev-train-sound-v1
## gs://kds-168dba29a1eb377402725ff4ec9b4a8afa032dfacaa1f1b84639cc59

Writing code/train_soundscape_binary_prediction.py


In [7]:
!python code/train_soundscape_binary_prediction.py

2022-05-24 03:40:38.768507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:40:38.907868: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:40:38.908727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:40:38.910581: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## now the bird prediction

In [8]:
%%writefile ./code/config/config.py


class config:
    EPOCHS = 20  # 20
    BATCH_SIZE = 16#32
    WAVE_LENGTH = 441000
    IMAGE_SIZE = -1
    IMG_FREQ = 128
    IMG_TIME = 862
    FOLD = 8#validation fold
    WEIGHT_SAVE = "comp_v4_SED"
    N_CLASSES = 152
    IS_COLAB  = True
    DATA_LINK = "gs://kds-a8fad3a0c9fb1420c6172cc9145a3232c612596be1f376e595a4c4ef"
    DATA_PATH = "data/tfrec/v3/"
    SEED = 1
    model_type = "resnext50_32x4d"
    sample_rate = 44100
    n_fft = 1024
    win_length = None
    hop_length = 512
    n_mels = 128
    LR_START = 0.000001  # 0.000001
    LR_MAX = 0.000005  # 0.00001
    LR_MIN = 0.0000001  # 0.000001
    LR_RAMP = 4
    SAVE_DIR = "../input/bird-clief-comp-v4-sed"
    RESUME = False
    RESUME_EPOCH = 0
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]


Overwriting ./code/config/config.py


In [9]:
%%writefile ./code/utils.py

import os
import re
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
from config.config import config


AUTO = tf.data.experimental.AUTOTUNE


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    #image = tf.image.resize(image, [config.IMAGE_SIZE//2, config.IMAGE_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    # normalization
    image = image - config.mean  # [0.485, 0.456, 0.406]
    image = image / config.std  # [0.229, 0.224, 0.225]
    return image




def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    #one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.strings.split(parts[-1], '.')[0]

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    image = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    image = tf.cast(image, tf.float32) / 255.0
    # normalization
    image = image - config.mean  # [0.485, 0.456, 0.406]
    image = image / config.std  # [0.229, 0.224, 0.225]
    return image

def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def get_eval_dataset():
    list_ds = tf.data.Dataset.list_files(str('./test_5sec_mel_spec/*'), shuffle=False)
    dataset = list_ds.map(process_path)
    dataset = dataset.repeat()  # the training dataset must repeat for several epochs
    #dataset = dataset.shuffle(2048)
    dataset = dataset.batch(config.BATCH_SIZE, drop_remainder=False, )# num_parallel_calls=AUTO
    #dataset = dataset.prefetch(
    #AUTO
    #)  # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


Overwriting ./code/utils.py


In [10]:
!ls

__notebook__.ipynb  binary_prediction.csv  code  test_5sec_mel_spec


In [11]:
%%writefile code/train_soundscape_comp_prediction.py
import sys
#sys.path.append('./code')
import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
from tqdm.auto import tqdm
import json
from datetime import datetime
from config.config import config
from utils import *
import shutil
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import gc
import tensorflow as tf
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
import time
print(gc.collect())

idx_to_bird = {118:'akiapo',
125:'aniani',
71:'apapan',
115:'barpet',
150:'crehon',
119:'elepai',
137:'ercfra',
99:'hawama',
101:'hawcre',
130:'hawgoo',
146:'hawhaw',
145:'hawpet1',
12:'houfin',
84:'iiwi',
53:'jabwar',
151:'maupar',
98:'omao',
147:'puaioh',
1:'skylar',
61:'warwhe1',
62:'yefcan'}



try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU ", tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)
config.BATCH_SIZE = config.BATCH_SIZE * strategy.num_replicas_in_sync

test_prediction = []
test_targets = []
test_file = []

## get the model for 8 fold

def model_get(model, save_dir, folds = 8):
    model_list = []
    for fold in range(0,folds):#8
        model_list.append(tf.keras.models.load_model(model))
        _=model_list[fold].load_weights(
        save_dir + f'/fold_{fold}/'+ "weights/" + "best.hdf5",
    )  ## work on the path
    return model_list

#for fold in range(0,10):
## model load
K.clear_session()
print('modle 1')
#model_list = model_get(model = "../input/birdclif-saved-model/comp_v4_SED",
#                      save_dir = "../input/bird-clief-comp-v4-sed")

print(gc.collect())
#model_list = model_list1 + model_list2 + model_list3
#model_list = model_list2 + model_list3

print('loaded teh models ')
test_dataset = get_eval_dataset()
actual_count = len(os.listdir('./test_5sec_mel_spec'))

test_prediction_fold = []

## model 1
test_prediction_sub_model_1_0 = []
test_prediction_sub_model_1_1 = []
test_prediction_sub_model_1_2 = []
test_prediction_sub_model_1_3 = []
test_prediction_sub_model_1_4 = []
test_prediction_sub_model_1_5 = []
test_prediction_sub_model_1_6 = []
test_prediction_sub_model_1_7 = []

## model 2
test_prediction_sub_model_2_0 = []
test_prediction_sub_model_2_1 = []
test_prediction_sub_model_2_2 = []
test_prediction_sub_model_2_3 = []
test_prediction_sub_model_2_4 = []
test_prediction_sub_model_2_5 = []
test_prediction_sub_model_2_6 = []
test_prediction_sub_model_2_7 = []

## model 3
test_prediction_sub_model_3_0 = []
test_prediction_sub_model_3_1 = []
test_prediction_sub_model_3_2 = []
test_prediction_sub_model_3_3 = []
test_prediction_sub_model_3_4 = []
test_prediction_sub_model_3_5 = []
test_prediction_sub_model_3_6 = []
test_prediction_sub_model_3_7 = []

## model 4
test_prediction_sub_model_4_0 = []
test_prediction_sub_model_4_1 = []
test_prediction_sub_model_4_2 = []
test_targets_fold = []
test_file_fold = []
#for rr, m in enumerate(model_list):
count = 0
#print('model ',rr)
test_prediction_sub = []
test_targets_sub = []
test_file_sub = []


model_list1 = model_get(model = "../input/birdclif-saved-model/comp_v4_SED",
                      save_dir = "../input/bird-clief-comp-v4-sed", folds = 3)

model_list2 = model_get(model = "../input/birdclif-saved-model/comp_v9_1_SED",
                      save_dir = "../input/bird-clief-comp-v9-1-sed", folds = 3)

model_list3 = model_get(model = "../input/birdclif-saved-model/comp_v8_1_SED",#convenxt
                      save_dir = "../input/bird-clief-comp-v8-1-sed", folds = 3)

model_list4 = model_get(model = "../input/birdclif-saved-model/comp_v8_SED",#convenxt
                      save_dir = "../input/bird-clief-comp-v8-sed/comp_v8_SED", folds = 3)

model_list = model_list1 + model_list2 + model_list3 + model_list4

for element in test_dataset:

    #pred = m.__call__(element[0])  # , verbose=0)
    #test_prediction_sub.append(pred)
    test_prediction_sub_model_1_0.append(model_list[0].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_1_1.append(model_list[1].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_1_2.append(model_list[2].__call__(element[0]).numpy().astype('float16'))
    
    test_prediction_sub_model_2_0.append(model_list[3].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_2_1.append(model_list[4].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_2_2.append(model_list[5].__call__(element[0]).numpy().astype('float16'))
    
    test_prediction_sub_model_3_0.append(model_list[6].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_3_1.append(model_list[7].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_3_2.append(model_list[8].__call__(element[0]).numpy().astype('float16'))
    
    test_prediction_sub_model_4_0.append(model_list[9].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_4_1.append(model_list[10].__call__(element[0]).numpy().astype('float16'))
    test_prediction_sub_model_4_2.append(model_list[11].__call__(element[0]).numpy().astype('float16'))
    
    test_file_sub.append(element[1].numpy())
    
    if count * config.BATCH_SIZE > actual_count + 20:  # this 20 is for safty
        print('the break ',count * config.BATCH_SIZE)
        break
    print('the out ',count * config.BATCH_SIZE)
    count += 1
    
del model_list
print(gc.collect())
print('going to sleep')
time.sleep(20)

test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_1_2)[:actual_count])


test_prediction_fold.append(np.concatenate(test_prediction_sub_model_2_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_2_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_2_2)[:actual_count])


test_prediction_fold.append(np.concatenate(test_prediction_sub_model_3_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_3_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_3_2)[:actual_count])

test_prediction_fold.append(np.concatenate(test_prediction_sub_model_4_0)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_4_1)[:actual_count])
test_prediction_fold.append(np.concatenate(test_prediction_sub_model_4_2)[:actual_count])

#test_targets_fold.append(np.concatenate(test_targets_sub)[:actual_count])
test_file_fold.append(np.concatenate(test_file_sub)[:actual_count])

test_prediction = np.array(test_prediction_fold).squeeze().mean(axis=0)
#test_targets = test_targets_fold[0]
test_file = test_file_fold[0]


oof_dict = {

    'itemid':test_file,
    #'hasbird':test_targets,

}

for i in range(0, config.N_CLASSES):
    oof_dict[i] = test_prediction[:,i]

oof_df = pd.DataFrame.from_dict(oof_dict)
oof_df['itemid'] =  oof_df['itemid'].map(lambda x: x.decode("utf-8")) 

## retain the unwanted birds
oof_df = oof_df.filter(items=list(idx_to_bird.keys())+['itemid'])

oof_df.to_csv(f"comp_prediction.csv", index = False)

## 5 sec
## happywhale-tfrev-train-sound-v2
## gs://kds-22c2720af882134066efd729205635efc3a6ce12dfa19de78bd41475

## 10 sec
## happywhale-tfrev-train-sound-v1
## gs://kds-168dba29a1eb377402725ff4ec9b4a8afa032dfacaa1f1b84639cc59

Writing code/train_soundscape_comp_prediction.py


In [12]:
#!sleep 20

In [13]:
!python code/train_soundscape_comp_prediction.py

0
REPLICAS:  1
modle 1
0
loaded teh models 
2022-05-24 03:44:20.438431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:44:20.474414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:44:20.475258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 03:44:20.476224: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, re

In [14]:
import gc
print(gc.collect())

273


In [15]:
#!sync; echo 3 > /proc/sys/vm/drop_caches

In [16]:
import numpy as np
np.array([1.0]).astype('float16').dtype


dtype('float16')

In [17]:
idx_to_bird = {118:'akiapo',
125:'aniani',
71:'apapan',
115:'barpet',
150:'crehon',
119:'elepai',
137:'ercfra',
99:'hawama',
101:'hawcre',
130:'hawgoo',
146:'hawhaw',
145:'hawpet1',
12:'houfin',
84:'iiwi',
53:'jabwar',
151:'maupar',
98:'omao',
147:'puaioh',
1:'skylar',
61:'warwhe1',
62:'yefcan'}

In [18]:
import pandas as pd
binary_df = pd.read_csv('./binary_prediction.csv')
comp_df = pd.read_csv('./comp_prediction.csv')
#comp_df.head()

bird_threshold = 0.4
ind_bird_threshold = 0.05
#binary_df = binary_df[binary_df['prediction']> bird_threshold]

binary_df = binary_df[binary_df['prediction']> binary_df['prediction'].quantile(0.10)]

comp1_df = comp_df[comp_df['itemid'].isin(binary_df['itemid'].values)]
for i in idx_to_bird.keys():
    comp1_df[str(i)] = comp1_df[str(i)]>comp_df[str(i)].median()#quantile(0.60)
non_comp_df = comp_df[~comp_df['itemid'].isin(binary_df['itemid'].values)]
for i in idx_to_bird.keys():
    non_comp_df[str(i)] = non_comp_df[str(i)]>1.1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [19]:
row_id_list = []
target_list = []

for index, row in comp1_df.iterrows():
    for i in idx_to_bird.items():
        row_id_list.append(row['itemid'].replace('sec',str(i[1])))
        target_list.append(row[str(i[0])])

for index, row in non_comp_df.iterrows():
    for i in idx_to_bird.items():
        row_id_list.append(row['itemid'].replace('sec',str(i[1])))
        target_list.append(row[str(i[0])])
        
submission_df = pd.DataFrame.from_dict({'row_id':row_id_list, 'target':target_list})
submission_df.to_csv('submission.csv',index=False)

In [20]:
submission_df

,row_id,target
0,soundscape_453028782_akiapo_10,True
1,soundscape_453028782_aniani_10,True
2,soundscape_453028782_apapan_10,False
3,soundscape_453028782_barpet_10,True
4,soundscape_453028782_crehon_10,False
...,...,...
247,soundscape_453028782_omao_45,False
248,soundscape_453028782_puaioh_45,False
249,soundscape_453028782_skylar_45,False
250,soundscape_453028782_warwhe1_45,False


In [21]:
#comp_df = pd.read_csv('./comp_prediction.csv')
#comp_df